In [1]:
# Access astronomical databases
from pyvo import registry  # version >=1.4.1 

# Moc and HEALPix tools
from mocpy import MOC

# Sky visualization
from ipyaladin import Aladin    # version >=0.3.0

# For plots
import matplotlib.pyplot as plt

# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

--------------------------------------------------------

Modify the <a href='https://vizier.cds.unistra.fr/'>VizieR catalogue</a> name (variable <b>CATALOGUE</b>) and anything else you might feel like exploring!

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1

**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [2]:
# the catalogue name in VizieR
CATALOGUE = "J/A+A/686/L2"

We first retrieve catalogue information.

In [3]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [4]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

Dormant 33 solar-masses black hole
Short Name: J/A+A/686/L2
IVOA Identifier: ivo://cds.vizier/j/a+a/686/l2
Access modes: tap#aux, web
Multi-capability service -- use get_service()

We announce the serendipitous discovery of a nearby (590pc) binary system
composed of an old very metal-poor giant star orbiting a black hole with an
estimated mass of 33M_{sun}_, in 11.6yr. The system was identified while
validating the preliminary Gaia astrometric binary solutions produced in
preparation for Gaia DR4, and was confirmed by Gaia RVS radial-velocity data.

Waveband Coverage:

Source: 2024A&A...686L...2G
Authors: Gaia Collaboration,  Panuzzo P.,  Mazeh T.,  Arenou F.,  Holl B. et al.
See creators attribute for the complete list of authors.
Alternative identifier(s): doi:10.26093/cds/vizier.36869002
More info: https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/686/L2


We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [5]:
voresource.creators[0]

'Gaia Collaboration'

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [6]:
tables = voresource.get_tables()
tables

{'J/A+A/686/L2/tableb1': <VODataServiceTable name="J/A+A/686/L2/tableb1">... 9 columns ...</VODataServiceTable>,
 'J/A+A/686/L2/tableb2': <VODataServiceTable name="J/A+A/686/L2/tableb2">... 5 columns ...</VODataServiceTable>}

In [7]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

['J/A+A/686/L2/tableb1', 'J/A+A/686/L2/tableb2']

The actual data can then be access using any of the ``access_modes`` of the voresource.

In [8]:
voresource.access_modes()

{'tap#aux', 'web'}

The web access is found by following the ``reference_url``

In [9]:
voresource.reference_url

'https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/686/L2'

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [10]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = voresource.get_service("tap").run_sync(
    f'select TOP 5 * from "{first_table_name}"',
)
tap_records

<DALResultsTable length=5>
recno     TransitID     CCDID    ObsTime     ... plxFactal     ScanPA    outFlag
                                    d        ...                deg             
int32       int64       int16    float64     ...  float64     float64     int16 
----- ----------------- ----- -------------- ... ---------- ------------ -------
    1 20114916805338633     1 2456958.110978 ... 0.70827985 -59.04672662       0
    2 20114916805338633     2 2456958.111034 ... 0.70827991 -59.04677105       0
    3 20114916805338633     3 2456958.111091 ... 0.70828003 -59.04681553       0
    4 20114916805338633     4 2456958.111156 ... 0.70828015 -59.04686711       0
    5 20114916805338633     5 2456958.111203 ... 0.70828021 -59.04690435       0

In [11]:
import pyvo as vo
service = vo.dal.TAPService("http://dc.g-vo.org/tap")

In [12]:
resultset = service.search("SELECT TOP 1 * FROM ivoa.obscore")

In [13]:
resultset

<DALResultsTable length=1>
dataproduct_type dataproduct_subtype ... source_table
                                     ...             
     object             object       ...    object   
---------------- ------------------- ... ------------
           image                     ... ppakm31.maps

#access modes Pyvo

In [14]:
for service in vo.regsearch(datamodel="obscore"):
  print(service['ivoid']) 

ivo://archive.stsci.edu/caomtap
ivo://astro.ucl.ac.uk/tap
ivo://astron.nl/tap
ivo://asu.cas.cz/tap
ivo://au.csiro/atoavo/tap
ivo://au.csiro/casda/tap
ivo://byu.arvo/tap
ivo://cadc.nrc.ca/argus
ivo://cefca/j-pas/j-pas-edr
ivo://cefca/j-plus/j-plus-dr1
ivo://cefca/j-plus/j-plus-dr2
ivo://cefca/j-plus/j-plus-dr3
ivo://cefca/jnep/j-nep-pdr202107
ivo://cefca/minijpas/minij-pas-pdr201912
ivo://cfa.tdc/tap
ivo://chivo/tap
ivo://eso.org/tap_obs
ivo://fai.kz/tap
ivo://fu-berlin.planet.hrsc/tap
ivo://jive.eu/tap
ivo://jvo/alma
ivo://jvo/isas/darts/akari/akari-irc_catalogue_allsky_astflux_1.0
ivo://jvo/isas/darts/akari/akari-irc_spectrum_pointed_diffuseskypatch_1.0
ivo://jvo/isas/darts/akari/akari-irc_spectrum_pointed_galpn_1.0
ivo://jvo/isas/darts/akari/akari-irc_spectrum_pointed_slitlessmir_spectrum_1.0
ivo://jvo/isas/darts/akari/obstap
ivo://jvo/isas/darts/halca/halca_vsop_correlated_data
ivo://jvo/isas/darts/halca/halca_vsop_survey_program_data
ivo://jvo/isas/darts/hitomi/hitomaster_v1
ivo://